In [ ]:
"""Complex Networks Analysis Project.

* @File    :   Project.ipynb
* @Time    :   2025/04/05 18:02:17
* @Authors :   Marc Ballestero Ribó and Arnau Jutglar Puig
* @Version :   0
* @Contact :   marcballestero@ub.edu, ajutglpu7@alumnes.ub.edu
* @License :   GNU GPL v3.0
* @Desc    :   None
"""


# Complex Networks Analysis Project: A Study of the Air Transportation Multiplex Dataset

This notebook is devoted to presenting all the code and reasonings of our project for the Complex Networks Analysis course of the Master in Fundamental Principles of Data Science.

Here, we analyze the "Air Transportation Multiplex" dataset, which contains a multiplex network of airline routes among European airports, where each of the 37 edge types represents routes by a different airline. It is an undirected, multiplex and unweighted network.

The notebook is organized in sections each one corresponding to a working week/period. Here's a summary of all the work done:

- **`04/04/2025 → 11/04/2025:`**
- **`11/04/2025 → 25/04/2025:`**
- **`25/04/2025 → 09/05/2025:`**
- **`09/04/2025 → 16/05/2025:`**
- **`16/05/2025 → 23/05/2025:`**
- **`23/05/2025 → 30/05/2025:`**

***